In [44]:
import pandas as pd

song_data = pd.read_csv('data/song_data_processed.csv')
eurovision = pd.read_csv('data/eurovision_processed.csv')
finals = pd.read_csv('data/finals_processed.csv')
ogae_polls = pd.read_csv('data/ogae_polls.csv')
world_polls = pd.read_csv('data/eurovisionworld_polls.csv')
points2016 = pd.read_csv('data/points_2016.csv')
points2017 = pd.read_csv('data/points_2017.csv')
points2018 = pd.read_csv('data/points_2018.csv')
points2019 = pd.read_csv('data/points_2019.csv')
points2021 = pd.read_csv('data/points_2021.csv')
points2022 = pd.read_csv('data/points_2022.csv')

In [78]:
counts = finals['Country'].value_counts().reset_index()
counts.columns = ['Country', 'Count']


In [96]:
# Column histogram: average placement by country ALL TIME
import pandas as pd
import plotly.express as px

data = finals[['Country','Place']]

grouped_data = data.groupby('Country').mean().reset_index()
grouped_data = grouped_data.sort_values(by=['Country']).reset_index()

counts = finals['Country'].value_counts().reset_index()
counts.columns = ['Country', 'Count']
counts = counts.sort_values(by=['Country']).reset_index()

data_df = pd.concat([grouped_data,counts['Count']],axis=1)
data_df = data_df.sort_values(by=['Place']).reset_index()

fig = px.scatter(data_df, x='Place', y='Count', hover_name='Country', 
                 title='Average Placement of Every Country in the Final and Numer of Times in the Final')
#fig.show()
name = "final_points_and_times_scatter.html"
fig.write_html(name) 


In [39]:
# Pie chart: average points (in percentages) in the final by year SINCE 2009
import plotly.express as px
import pandas as pd

song_data = song_data.dropna()
years = song_data.groupby('year')
keys = list(years.groups)

for key in keys:
    year = years.get_group(key)
    pie_title = "Distribution of Points in Final in " + str(key)
    fig = px.pie(year, values='final_total_points', names='country', title=pie_title)
    #fig.show()
    name = "final_points_pie_year/final_points_pie_year_" + str(key) + ".html"
    fig.write_html(name) 

fig = px.pie(song_data, values='final_total_points', names='country', title=pie_title)
name = "final_points_pie_year/final_points_pie_year_All.html"
fig.write_html(name) 

In [14]:
# Pie chart: style of the song by country and year (in percentages) SINCE 2009
import plotly.express as px
import pandas as pd

years = list(range(2009,2023))

for year in years:
    df = song_data.loc[song_data['year'] == year]
    counts = df['style'].value_counts().reset_index()
    counts.columns = ['style', 'count']
    pie_title = "Style Distribution of Songs in " + str(year)
    fig = px.pie(counts, values='count', names='style', title=pie_title)
    #fig.show()
    name = "style_pie_year/style_pie_year_" + str(year) + ".html"
    fig.write_html(name) 

countries = list(song_data['country'].unique())
countries = [x for x in countries if str(x) != 'nan']
countries = sorted(countries)

for country in countries:
    df = song_data.loc[song_data['country'] == country]
    counts = df['style'].value_counts().reset_index()
    counts.columns = ['style', 'count']
    pie_title = "Style Distribution of Songs of " + str(country)
    fig = px.pie(counts, values='count', names='style', title=pie_title)
    #fig.show()
    name = "style_pie_country/style_pie_country_" + str(country) + ".html"
    fig.write_html(name) 

counts = song_data['style'].value_counts().reset_index()
counts.columns = ['style', 'count']
pie_title = "Style Distribution of All Songs"
fig = px.pie(counts, values='count', names='style', title=pie_title)
#fig.show()
name = "style_pie_year/style_pie_year_All.html"
fig.write_html(name) 

name = "style_pie_country/style_pie_country_All.html"
fig.write_html(name)

In [8]:
# Pie chart: language used by country and year (in percentages) SINCE 2009
import plotly.express as px
import pandas as pd

years = list(range(2009,2023))

for year in years:
    df = song_data.loc[song_data['year'] == year]
    counts = df['language'].value_counts().reset_index()
    counts.columns = ['language', 'count']
    pie_title = "Language Distribution of Songs in " + str(year)
    fig = px.pie(counts, values='count', names='language', title=pie_title)
    #fig.show()
    name = "language_pie_year/language_pie_year_" + str(year) + ".html"
    fig.write_html(name) 

countries = list(song_data['country'].unique())
countries = [x for x in countries if str(x) != 'nan']
countries = sorted(countries)

for country in countries:
    df = song_data.loc[song_data['country'] == country]
    counts = df['language'].value_counts().reset_index()
    counts.columns = ['language', 'count']
    pie_title = "Language Distribution of Songs of " + str(country)
    fig = px.pie(counts, values='count', names='language', title=pie_title)
    #fig.show()
    name = "language_pie_country/language_pie_country_" + str(country) + ".html"
    fig.write_html(name) 


In [33]:
# Pie chart: female / male as representative by country and year (in percentages) and All SINCE 2009

import plotly.express as px
import pandas as pd

years = list(range(2009,2023))

for year in years:
    df = song_data.loc[song_data['year'] == year]
    counts = df['gender'].value_counts().reset_index()
    counts.columns = ['gender', 'count']
    pie_title = "Gender Distribution of Songs in " + str(year)
    fig = px.pie(counts, values='count', names='gender', title=pie_title)
    #fig.show()
    name = "gender_pie_year/gender_pie_year_" + str(year) + ".html"
    fig.write_html(name) 

countries = list(song_data['country'].unique())
countries = [x for x in countries if str(x) != 'nan']
countries = sorted(countries)

for country in countries:
    df = song_data.loc[song_data['country'] == country]
    counts = df['gender'].value_counts().reset_index()
    counts.columns = ['gender', 'count']
    pie_title = "Gender Distribution of Songs of " + str(country)
    fig = px.pie(counts, values='count', names='gender', title=pie_title)
    #fig.show()
    name = "gender_pie_country/gender_pie_country_" + str(country) + ".html"
    fig.write_html(name) 

counts = song_data['gender'].value_counts().reset_index()
counts.columns = ['gender', 'count']
pie_title = "Gender Distribution of All Songs"
fig = px.pie(counts, values='count', names='gender', title=pie_title)
#fig.show()
name = "gender_pie_year/gender_pie_year_All.html"
fig.write_html(name) 

name = "gender_pie_country/gender_pie_country_All.html"
fig.write_html(name)